In [1]:
import deepxde as dde
import numpy as np

RuntimeError: Backend is set as tensorflow.compat.v1, but 'tensorflow.compat.v1' failed to import.

In [ ]:
def func(x):
    """
    x: array_like, N x D_in
    y: array_like, N x D_out
    """
    return x * np.sin(5 * x)

In [ ]:
geom = dde.geometry.Interval(-1, 1)

In [ ]:
num_train = 16
num_test = 100
data = dde.data.Function(geom, func, num_train, num_test)

In [ ]:
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN([1] + [20] * 3 + [1], activation, initializer)

In [ ]:
model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])

In [ ]:
losshistory, train_state = model.train(iterations=10000)

In [2]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

NameError: name 'dde' is not defined